In [ ]:
%cd /content/drive/MyDrive/ri/code/data

In [ ]:
import re

In [ ]:
import itertools

In [ ]:
import nltk

In [ ]:
from nltk.corpus import stopwords

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer

In [ ]:
import pandas as pd

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
from collections import Counter

In [ ]:
#Data Understanding and Preparation

In [ ]:
## Import Libraries and Load Data

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ri/code/data/A1_standard.csv')

In [ ]:
print(df.head())

In [ ]:
## Data Information and Missing Values

In [ ]:
df.info()

In [ ]:
import seaborn as sns

In [ ]:
import matplotlib.pyplot as plt